#  Мой психолог ИИ помоги нейросети выявить уровень стресса.







Набор данных представляет из себя зарегистрированные при помощи датчиков реакции на вопросы о факторах риска на проверках при трудоустройстве. Они размеченны специалистами по трем уровням стресса (слабый, средний, сильный).  Каждая проверка представляет из себя несколько групп вопросов, каждый из которых задается не более 4 раз. Для каждого человека была следующая структура тестирования:

Группа1 Повторение1: Вопрос1, Вопрос2, ..., Вопрос12
Группа1 Повторение2: ВопросI1, ВопросI2, ..., ВопросI12 (где I1, I2, ..., I12 - вопросы из Повторения1 в случайном порядке)
Группа1 Повторение3: ВопросI1, ВопросI2, ..., ВопросI12 (где I1, I2, ..., I12 - вопросы из Повторения1 в случайном порядке)
Группа2 Повторение1: Вопрос1, Вопрос2, ..., Вопрос12
Группа2 Повторение2: ВопросI1, ВопросI2, ..., ВопросI12 (где I1, I2, ..., I12 - вопросы из Повторения2 в случайном порядке)
Группа2 Повторение3: ВопросI1, ВопросI2, ..., ВопросI12 (где I1, I2, ..., I12 - вопросы из Повторения2 в случайном порядке)
и так далее ...

Количество групп от 1 до 7, в группах может быть от 1 до 14 вопросов. 

Соответственно, в наборе данных присутствуют:

* `Filename` - уникальный id испытуемого.
* `Test_index` - номер группы, нумерация с 0
* `Presentation` - номер повторения, нумерация с 1
* `Question` - номер вопроса, нумерация с 0
* `Data` - равномерно зарегистрированные полиграфом 12 секунд сигнала с частотой 20 Гц (240 значений) (метка датчика регистрации сигнала - фотоплетизмограмма)
* `Data_2` - равномерно зарегистрированные полиграфом 12 секунд сигнала с частотой 20 Гц (240 значений) (метка датчика регистрации сигнала - пьезоплетизмограмма)
* `Class_label  Data_type` - метка класса (0 - слабый стресс, 1 - средний стресс, 2 - сильный стресс)

Важное примечание: согласно методике полиграфолог при выставлении баллов оценивает реакции на вопросы при помощи попарного сравнения в рамках каждого повторения отдельно - не более 14 вопросов. Такая процедура выполняется независимо по каждой группе вопросов по каждому повторению.

Нужно:

1. Загрузить данные 
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами.
4. Проверить данные на тестовой выборке и сделать выводы.


## Подготовка

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [2]:
# блок импорта
import pandas as pd


# Подготовка моделей
from sklearn.model_selection import train_test_split



# Метрики
from sklearn.metrics import mean_absolute_error, accuracy_score


from sklearn.preprocessing import StandardScaler


import re
from catboost import CatBoostClassifier
from matplotlib import pyplot as plt

In [3]:
data = pd.read_excel('dataset_train.xlsx')
data_test =  pd.read_csv('dataset_test.csv', sep=';')

In [4]:
def clean_data_1(row):
  i = re.sub('[^\d\,-]', '', row['Data'])
  return i

def clean_data_2(row):
  i = re.sub('[^\d\,-]', '', row['Data_2'])
  return i

In [5]:
data['Data'] = data.apply(clean_data_1, axis=1)
features_1 = data['Data'].str.split(',',expand=True)

In [6]:
data['Data_2'] = data.apply(clean_data_2, axis=1)
features_2 = data['Data_2'].str.split(',',expand=True)

## Анализ

In [7]:
data_features = pd.concat([features_1, features_2], axis=1)

In [8]:
new_col = list(map(str,range(480)))
data_features = data_features.set_axis(new_col, axis=1, inplace=False)

In [9]:
data_full = pd.concat([data, data_features],axis=1).drop(['Data','Data_2'], axis =1)

In [10]:
#убираем пропуски в данных
data_full = data_full.dropna(axis=0).reset_index(drop=True)

In [11]:
data_full['median_1'] = data_full.iloc[:, 5:245].median(axis=1)
data_full['median_2'] = data_full.iloc[:, 245:485].median(axis=1)

In [12]:
data_full = data_full.query('-1000 < median_1 <500 and -1000< median_2 < 1000')

In [13]:
data_full.iloc[:, 5:245] = data_full.iloc[:, 5:245].astype('int')
data_full.iloc[:, 245:485] = data_full.iloc[:, 245:485].astype('int')

In [14]:
#убираем пропуски в данных
data_full = data_full.dropna(axis=0).reset_index(drop=True)

Масштабируем признаки

In [15]:
#масштабирую данные фотоплетизмограммы
scaler = StandardScaler()
model = scaler.fit(data_full.iloc[:, 5:245])
data_full.iloc[:, 5:245] = model.transform(data_full.iloc[:, 5:245])

In [16]:
#масштабирую данные пьезоплетизмограммы
scaler = StandardScaler()
model = scaler.fit(data_full.iloc[:, 245:485])
data_full.iloc[:, 245:485] = model.transform(data_full.iloc[:, 245:485])

## Обучение

In [17]:
#создаём категориальные признаки
data_full[['Test_index', 'Presentation', 'Question']] = data_full[['Test_index', 'Presentation', 'Question']].astype("category")

In [18]:
features = data_full.drop(['Filename','Class_label'], axis =1)
target = data_full['Class_label']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.25, random_state = 12345)

In [19]:
categorial = ['Test_index', 'Presentation', 'Question', 'Question']

In [20]:
def train_cb():
    default_parameters = {
        "custom_metric": ['MultiClass', 'AUC:hints=skip_train~false'],
        "random_seed": 42,
        "loss_function": "MultiClass",
        "metric_period": 500,
        "depth": 6,
        "bootstrap_type": "Bayesian",
        "bagging_temperature": 0.66

    }

    parameters = {
        'iterations': 2500,
        'task_type': "CPU",
        **default_parameters
    }

    clf = CatBoostClassifier(**parameters, silent=True, cat_features=categorial)
    clf.fit(features_train, target_train, verbose=500, plot=False)
    return clf

 

In [21]:
print("Fitting model...")
model_cb = train_cb()

Fitting model...
Learning rate set to 0.042614
0:	learn: 1.0886365	total: 306ms	remaining: 12m 45s
500:	learn: 0.8674243	total: 1m 9s	remaining: 4m 38s
1000:	learn: 0.7624684	total: 2m 14s	remaining: 3m 20s
1500:	learn: 0.6731499	total: 3m 34s	remaining: 2m 22s
2000:	learn: 0.5966498	total: 4m 59s	remaining: 1m 14s
2499:	learn: 0.5316068	total: 6m 4s	remaining: 0us


In [27]:
predicts_CB = model_cb.predict(features_test)

In [28]:
MAE_CB = mean_absolute_error(target_test, predicts_CB)
print('MAE tree', MAE_CB)
ACC_CB = accuracy_score(target_test, predicts_CB)
print('ACC tree', ACC_CB)

MAE tree 0.445141065830721
ACC tree 0.5618409803362782


## Тестирование

In [29]:
def predict(data):
  data_test = data.copy()
  data_test = data_test.drop(['id'], axis =1)

  data_test['Data'] = data_test.apply(clean_data_1, axis=1)
  features_1 = data_test['Data'].str.split(',',expand=True)
  data_test['Data_2'] = data_test.apply(clean_data_2, axis=1)
  features_2 = data_test['Data'].str.split(',',expand=True)

  data_features = pd.concat([features_1, features_2],axis=1)

  new_col = list(map(str,range(480)))
  data_features = data_features.set_axis(new_col, axis=1, inplace=False)

  data_full = pd.concat([data_test, data_features],axis=1).drop(['Data','Data_2'], axis =1)

  data_full.iloc[:, 5:245] = data_full.iloc[:, 5:245].astype('int')
  data_full.iloc[:, 245:485] = data_full.iloc[:, 245:485].astype('int')

  data_full['median_1'] = features_1.iloc[:, :-1].median(axis=1)
  data_full['median_2'] = features_2.iloc[:, :-1].median(axis=1)

  scaler = StandardScaler()
  model = scaler.fit(data_full.iloc[:, 5:245])
  data_full.iloc[:, 5:245] = model.transform(data_full.iloc[:, 5:245])
  model = scaler.fit(data_full.iloc[:, 245:485])
  data_full.iloc[:, 245:485] = model.transform(data_full.iloc[:, 245:485])

  #predict
  data_full[['Test_index', 'Presentation', 'Question']] = data_full[['Test_index', 'Presentation', 'Question']].astype("category")
  features = data_full.drop(['Filename','Class_label'], axis =1)

  #categorial = ['Test_index', 'Presentation', 'Question']

  predicts_CB = model_cb.predict(features)
  predicts_CB = pd.Series(predicts_CB.flatten())
  result = pd.DataFrame(data['id'])
  result = pd.concat([result, predicts_CB], axis =1).reset_index(drop=True)

  return(result)

In [30]:
test = predict(data_test)

C:\anaconda3\envs\PyTorchEnv\lib\site-packages\ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
C:\anaconda3\envs\PyTorchEnv\lib\site-packages\ipykernel_launcher.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [31]:
col = ['id', 'Class_label']
test_result = test.set_axis(col,axis = 1,  inplace=False)

In [ ]:
from google.colab import files

In [ ]:
test_result.info()

In [32]:
test_result.to_csv('never_gonna_give_you_up.csv', sep=';', index=False) 
files.download('never_gonna_give_you_up.csv')

NameError: name 'files' is not defined